In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as K
import matplotlib.pyplot as plt

In [2]:
np.random.seed(11)
tf.random.set_seed(11)
batch_size = 256
max_epochs = 50
learning_rate = 1e-3
momentum = 8e-1
hidden_dim = 128
original_dim = 784

In [3]:
import io
import os
import json
import pandas as pd
import numpy as np

os.chdir("E:/Kakao Arena")

def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)

    return json_obj

song = pd.DataFrame(load_json("song_meta.json"))
val = pd.DataFrame(load_json("val.json"))
import pickle
with open("train_genre.pickle","rb")as fr:
    train_genre = pickle.load(fr)
train_genre.head()

,id,like_cnt,plylst_title,songs,tags,updt_date,genre
0,61281,71,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",[락],2013-12-19 18:36:19.000,"[J-POP, 일렉트로니카, R&B/Soul, POP, 포크/블루스/컨트리, 록/메탈]"
1,10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[추억, 회상]",2014-12-02 16:19:42.000,"[포크/블루스, 일렉트로니카, 재즈, 뉴에이지, POP, OST, 발라드, 록/메탈..."
2,76951,17,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[까페, 잔잔한]",2017-08-28 07:09:34.000,"[포크/블루스, 랩/힙합, 재즈, 일렉트로니카, R&B/Soul, 발라드, 록/메탈..."
3,147456,33,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",2019-12-05 15:15:18.000,"[성인가요, 포크/블루스, 재즈, 어린이/태교, POP, R&B/Soul, 뉴에이지..."
4,27616,9,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",[댄스],2011-10-25 13:54:56.000,"[랩/힙합, R&B/Soul, 발라드, 댄스, 록/메탈, 아이돌]"


In [5]:
song["id"] 

0              0
1              1
2              2
3              3
4              4
           ...  
707984    707984
707985    707985
707986    707986
707987    707987
707988    707988
Name: id, Length: 707989, dtype: int64

In [6]:
aa = train_genre["songs"].tolist()

In [7]:
from collections import Counter
tmp_order = [y for x in aa for y in x]
Counter(tmp_order).values()

dict_values([7, 3, 1, 15, 57, 24, 2, 46, 40, 3, 1, 1, 4, 7, 1, 63, 1, 1, 3, 54, 45, 1111, 15, 311, 38, 135, 34, 18, 853, 417, 204, 102, 18, 29, 37, 48, 30, 20, 30, 12, 11, 8, 3, 1103, 1207, 105, 197, 77, 2, 1, 2, 390, 4, 8, 7, 23, 26, 24, 43, 244, 84, 188, 84, 363, 17, 39, 23, 9, 101, 20, 14, 238, 72, 52, 41, 30, 22, 26, 39, 9, 31, 23, 99, 18, 15, 113, 134, 54, 46, 181, 237, 42, 11, 46, 12, 12, 141, 386, 839, 78, 1103, 15, 221, 1215, 27, 33, 28, 444, 13, 164, 61, 199, 4, 20, 62, 277, 58, 485, 45, 209, 86, 191, 106, 150, 18, 1186, 159, 51, 152, 72, 219, 430, 136, 238, 415, 262, 63, 203, 148, 16, 134, 178, 165, 597, 551, 93, 38, 173, 29, 29, 296, 3, 502, 350, 21, 7, 125, 7, 229, 452, 113, 33, 106, 383, 174, 297, 235, 261, 197, 196, 73, 297, 113, 275, 45, 102, 146, 201, 23, 62, 683, 840, 264, 327, 159, 52, 72, 39, 222, 4, 54, 346, 13, 27, 381, 60, 10, 16, 41, 35, 712, 408, 2, 2, 33, 1, 2, 1306, 3, 65, 400, 1334, 6, 1424, 1354, 259, 307, 140, 138, 121, 119, 556, 590, 1468, 309, 430, 90, 9,

In [8]:
from collections import Counter
pd.Series(list(Counter(tmp_order).values())).describe()

count    615142.000000
mean          8.592928
std          39.481912
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max        2175.000000
dtype: float64

In [9]:
c = Counter(tmp_order)
count_df = pd.DataFrame(data=c.values(),index=c.keys(),columns=["count"])
# 평균이 8이라서 count가 8 이상인 것만 뽑기
count_df = count_df[count_df["count"] > 20]

In [10]:
song_list = list(count_df.index)
len(song_list)

42109

In [11]:
train_genre["songs"]

0         [525514, 129701, 383374, 562083, 297861, 13954...
1         [432406, 675945, 497066, 120377, 389529, 24427...
2         [83116, 276692, 166267, 186301, 354465, 256598...
3         [394031, 195524, 540149, 287984, 440773, 10033...
4         [159327, 553610, 5130, 645103, 294435, 100657,...
                                ...                        
115066    [429629, 441511, 612106, 516359, 691768, 38714...
115067    [321330, 216057, 534472, 240306, 331098, 23288...
115068    [50512, 249024, 250608, 371171, 229942, 694943...
115069    [533534, 608114, 343608, 417140, 609009, 30217...
115070    [26008, 456354, 324105, 89871, 135272, 143548,...
Name: songs, Length: 115071, dtype: object

In [12]:
import copy
tmp_genre = copy.deepcopy(train_genre["songs"])
tmp_genre

0         [525514, 129701, 383374, 562083, 297861, 13954...
1         [432406, 675945, 497066, 120377, 389529, 24427...
2         [83116, 276692, 166267, 186301, 354465, 256598...
3         [394031, 195524, 540149, 287984, 440773, 10033...
4         [159327, 553610, 5130, 645103, 294435, 100657,...
                                ...                        
115066    [429629, 441511, 612106, 516359, 691768, 38714...
115067    [321330, 216057, 534472, 240306, 331098, 23288...
115068    [50512, 249024, 250608, 371171, 229942, 694943...
115069    [533534, 608114, 343608, 417140, 609009, 30217...
115070    [26008, 456354, 324105, 89871, 135272, 143548,...
Name: songs, Length: 115071, dtype: object

In [ ]:
from multiprocessing import Pool, freeze_support
import time

def for_encoding(i):
    
    lst=[]
    
    for j in range(len(song_list)):
        if song_list[j] not in practice[i]:
            lst.append(0)
        else:
            lst.append(1)
            
    if i%100==0:
        print(i)
    if i%1000==0:
        time.sleep(3)
    
    return lst
        
if __name__ == '__main__':
    start_time = time.time()
    freeze_support()
    with Pool(processes=15) as p:
        for_encod = p.map(for_encoding, range(len(tmp_genre)))
    print("--- %s seconds ---" % (time.time()-start_time))

# 윈도우에서 병렬처리 안돼서 우분투에서 돌려서가져옴

In [4]:
for_encod = pd.read_csv('for_encod.csv',header=None)
for_encod

,0,1,2,3,4,5,6,7,8,9,...,14807,14808,14809,14810,14811,14812,14813,14814,14815,14816
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115066,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
115067,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
115068,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
115069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
len(for_encod.columns)

14817

In [6]:
tt = np.asarray(for_encod)[0:500000]

In [8]:
x = tf.keras.Input(shape=(len(for_encod.columns),))

assert x.shape.as_list() == [None,len(for_encod.columns)]

encoded = tf.keras.layers.Dense(2000,activation='relu')(x)
decoded = tf.keras.layers.Dense(len(for_encod.columns),activation='sigmoid')(encoded)

autoencoder = tf.keras.models.Model(x,decoded)

encoder = tf.keras.models.Model(x, encoded)

encoded_input = tf.keras.layers.Input(shape=(2000,))

assert encoded_input.shape.as_list() == [None,2000]

decoder_layer = autoencoder.layers[-1]
decoder = tf.keras.models.Model(encoded_input, decoder_layer(encoded_input))
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

#sp_x = tf.sparse.reorder(sparse_tensor)
#tf_x = tf.sparse.to_dense(sp_x)
autoencoder.fit(tt, tt, epochs=50, batch_size=100, shuffle=True )


MemoryError: Unable to allocate 12.7 GiB for an array with shape (115071, 14817) and data type int64

In [28]:
encoded_tmp = encoder.predict(tt)
decoded_tmp = decoder.predict(encoded_tmp)
decoded_tmp

array([[0.49990845, 0.49998423, 0.50007915, ..., 0.49993548, 0.4999982 ,
        0.50000703],
       [0.49953547, 0.5000437 , 0.5005251 , ..., 0.5001735 , 0.4994905 ,
        0.4995853 ],
       [0.4998483 , 0.49998063, 0.50013435, ..., 0.5001229 , 0.4997822 ,
        0.49994254],
       ...,
       [0.49999663, 0.49999663, 0.49999747, ..., 0.49999577, 0.4999958 ,
        0.49999577],
       [0.49975902, 0.49985763, 0.50014526, ..., 0.49978116, 0.49972898,
        0.5000537 ],
       [0.4999966 , 0.49999663, 0.4999975 , ..., 0.49999574, 0.49999583,
        0.49999583]], dtype=float32)

In [46]:
decoded_tmp[0]

array([0.49990845, 0.49998423, 0.50007915, ..., 0.49993548, 0.4999982 ,
       0.50000703], dtype=float32)

In [58]:
topid = []
for i in range(len(decoded_tmp)):
    topid.append(sorted(range(len(decoded_tmp[i])),key= lambda j: decoded_tmp[i][j],reverse=True)[:200])
topid

[5849,
 10378,
 10880,
 12766,
 10073,
 13995,
 6526,
 8853,
 12402,
 14088,
 3129,
 3904,
 4149,
 4880,
 350,
 11925,
 8967,
 3526,
 11021,
 6729,
 7522,
 10815,
 10612,
 3857,
 12712,
 9424,
 12945,
 14319,
 443,
 1409,
 9589,
 10578,
 8057,
 10346,
 9948,
 4591,
 585,
 3517,
 3698,
 1528,
 9556,
 4164,
 13619,
 7195,
 8707,
 4034,
 10731,
 13167,
 8696,
 11480,
 992,
 13569,
 9101,
 3181,
 11749,
 11719,
 12563,
 13911,
 8349,
 852,
 11059,
 763,
 2021,
 3428,
 1688,
 7916,
 6766,
 1577,
 2594,
 6555,
 13984,
 11181,
 1439,
 13115,
 7296,
 3532,
 1723,
 13674,
 13284,
 5179,
 14498,
 4969,
 6391,
 11476,
 5224,
 11077,
 14702,
 5072,
 11200,
 5201,
 6331,
 9190,
 5856,
 587,
 3604,
 3176,
 10654,
 6837,
 7144,
 12472,
 11336,
 8611,
 10270,
 14007,
 3208,
 4385,
 8570,
 8816,
 1176,
 6009,
 8842,
 8583,
 4437,
 30,
 9938,
 7465,
 14298,
 5538,
 12078,
 1480,
 10366,
 11152,
 5270,
 5494,
 6055,
 8661,
 6397,
 12162,
 1782,
 7481,
 10627,
 4463,
 5013,
 40,
 1428,
 9098,
 487,
 5842,

In [44]:
len(for_encod.columns[decoded_tmp[0]>0.5])

6889

In [39]:
for_encod.columns

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            14807, 14808, 14809, 14810, 14811, 14812, 14813, 14814, 14815,
            14816],
           dtype='int64', length=14817)